<a href="https://colab.research.google.com/github/SridharSeshadri56/Decision_Models/blob/main/pyomoMaxFlow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Sets up a max flow problem on a given network. Data are nodes, arcs, capacities of arcs. 
# Objective is to maximize the Net Flow to the demand node. (If there are many demand nodes one can connect them to a dummy node 
# and maximize flow)

In [1]:
pip install pyomo

     |████████████████████████████████| 9.6 MB 11.9 MB/s 
     |████████████████████████████████| 49 kB 4.1 MB/s 


In [2]:
!apt-get install -y -qq glpk-utils  #Installs the optimization engine called glpk.

Selecting previously unselected package libsuitesparseconfig5:amd64.
(Reading database ... 155320 files and directories currently installed.)
Preparing to unpack .../libsuitesparseconfig5_1%3a5.1.2-2_amd64.deb ...
Unpacking libsuitesparseconfig5:amd64 (1:5.1.2-2) ...
Selecting previously unselected package libamd2:amd64.
Preparing to unpack .../libamd2_1%3a5.1.2-2_amd64.deb ...
Unpacking libamd2:amd64 (1:5.1.2-2) ...
Selecting previously unselected package libcolamd2:amd64.
Preparing to unpack .../libcolamd2_1%3a5.1.2-2_amd64.deb ...
Unpacking libcolamd2:amd64 (1:5.1.2-2) ...
Selecting previously unselected package libglpk40:amd64.
Preparing to unpack .../libglpk40_4.65-1_amd64.deb ...
Unpacking libglpk40:amd64 (4.65-1) ...
Selecting previously unselected package glpk-utils.
Preparing to unpack .../glpk-utils_4.65-1_amd64.deb ...
Unpacking glpk-utils (4.65-1) ...
Setting up libsuitesparseconfig5:amd64 (1:5.1.2-2) ...
Setting up libcolamd2:amd64 (1:5.1.2-2) ...
Setting up libamd2:amd64 

In [3]:
from pyomo.environ import *
nodes = ['Stuttgart', 'Rotterdam', 'Bordeaux', 'Lisbon', 'NewYork', 'NewOrleans', 'LosAngeles']     #The nodes in the network

#The arcs in the network which will serve to define decision variables
arcs = { ('Stuttgart', 'Rotterdam'), ('Stuttgart','Bordeaux'), ('Stuttgart', 'Lisbon'), \
       ('Rotterdam', 'NewYork'), ('Bordeaux', 'NewYork'), ('Bordeaux', 'NewOrleans'), \
       ('Lisbon','NewOrleans'), ('NewYork', 'LosAngeles'), ('NewOrleans', 'LosAngeles')}

     #Max flow on arcs  
arc_capacity =  { ('Stuttgart', 'Rotterdam'):50, ('Stuttgart','Bordeaux'):70, ('Stuttgart', 'Lisbon'):40, \
       ('Rotterdam', 'NewYork'):60, ('Bordeaux', 'NewYork'):40, ('Bordeaux', 'NewOrleans'):50, \
       ('Lisbon','NewOrleans'):30, ('NewYork', 'LosAngeles'):80, ('NewOrleans', 'LosAngeles'):70}

 # Demand > 0 means supply node, < 0 means demand, = 0 means transhipment node
demand = { 'Stuttgart':1000, 'Rotterdam':0, 'Bordeaux':0, 'Lisbon':0, 'NewYork':0, 'NewOrleans':0, 'LosAngeles':0 }
# Note I set demand = 0 at Los Angeles. So it can take any amount of inflow.Recall inflow is negative.
# I set supply to be large (1000) at Stuttgart so it does not limit the solution

model = ConcreteModel(name = "(Model2)")                #Object to define the model (any name on left hand side ok)

model.x = Var( arcs, within= NonNegativeReals )         #The decision variables are flows in the arcs

model.value = Objective(
expr = sum(- model.x[i,j] for (i,j) in arcs if i == 'LosAngeles') + \
    sum(  model.x[i,j] for (i, j) in arcs if j == 'LosAngeles'), sense = maximize )  # Maximize net flow to Los Angeles
# Note it is in form inflows minus outflow = net flow


def one_per_node(m,c):                          # For any node outflow - inflow <= demand
    return sum(m.x[i,j] for (i,j) in arcs if i == c) - sum(m.x[i,j] for (i,j) in arcs if j == c) <= demand [c]

# Limit of arc capacity
def one_per_arc(m,c1, c2):
    return m.x[c1, c2] <= arc_capacity[c1, c2]

model.one_per_node = Constraint(nodes, rule = one_per_node)   #Net flow should be less than or equal to demand
model.one_per_arc  = Constraint(arcs, rule = one_per_arc)     #Max flow on arc can not exceed arc capacity

opt = SolverFactory('glpk')

model.dual = Suffix(direction=Suffix.IMPORT_EXPORT)
results = opt.solve(model, tee= True)

GLPSOL: GLPK LP/MIP Solver, v4.65
Parameter(s) specified in the command line:
 --write /tmp/tmpwfxi8ral.glpk.raw --wglp /tmp/tmpv_nnnre1.glpk.glp --cpxlp
 /tmp/tmpx2fcuvgs.pyomo.lp
Reading problem data from '/tmp/tmpx2fcuvgs.pyomo.lp'...
17 rows, 10 columns, 28 non-zeros
98 lines were read
Writing problem data to '/tmp/tmpv_nnnre1.glpk.glp'...
77 lines were written
GLPK Simplex Optimizer, v4.65
17 rows, 10 columns, 28 non-zeros
Preprocessing...
5 rows, 9 columns, 13 non-zeros
Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00
Problem data seem to be well scaled
Constructing initial basis...
Size of triangular part is 5
*     0: obj =  -0.000000000e+00 inf =   0.000e+00 (2)
*    10: obj =   1.500000000e+02 inf =   0.000e+00 (0)
OPTIMAL LP SOLUTION FOUND
Time used:   0.0 secs
Memory used: 0.0 Mb (40847 bytes)
Writing basic solution to '/tmp/tmpwfxi8ral.glpk.raw'...
36 lines were written


In [ ]:
model.pprint()